Рекомендательные системмы являются одними из самых популярных приложений DS. Они используются для прогнозирования предпочтений,  которые пользователь поставит какому-нибудь товару в магазине. Amazon использует их, чтобы предлагать продукты клиентам, YouTube решает какое видео воспроизводить следующим, а Facebook рекомендует посты. Существуют также рекомендательные системы для таких доменов, как рестораны, фильмы и онлайн-знакомства. Более того, для некоторых компаний, таких как Netflix, бизнес-модель и ее успех зависят от эффективности их рекомендаций. Netflix даже предложил миллион долларов в 2009 году каждому, кто сможет улучшить его систему рекомендаций на 10%.

Вообще говоря, рекомендательные системы могут быть поделены на 3 типа:
* Базовый подход: предлагайте общие рекомендации каждому пользователю в зависимости от популярности фильма и / или жанра. Например, Кинопоиск 250.
* Content-based рекомендательные системы. Эта система использует метаданные элемента, например жанр, режиссер, описание, актеры и т.д. для фильмов. Общая идея, лежащая в основе этих рекомендательных систем, заключается в том, что если человеку нравится конкретный предмет, ему также понравится предмет, похожий на него. Алгоритм будет использовать метаданные прошлых элементов пользователя. Хорошим примером может служить YouTube, где на основе вашей истории он предлагает вам новые видео, которые вы потенциально можете посмотреть.
* Коллаборативная фильтрация. Эти системы пытаются предсказать рейтинг, который пользователь поставит элементу, на основе прошлых оценок и предпочтений других пользователей. Коллаборативной фильтрации не требуются метаданные элемента.

# Простые рекомендации

* Сначала нужно определить метрику, по которой будем сортировать фильмы
* Затем посчитать метрику для каждого фильма
* Отсортировать фильмы, и выбрать топ результатов

Для работы будем использовать популярный датасет MovieLens. Датасет можно скачать по [ссылке](https://www.kaggle.com/rounakbanik/the-movies-dataset). 

In [1]:
%load_ext autoreload
%autoreload 2

from src.data.dataset import get_metadata

metadata = get_metadata()

metadata

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


Одна из базовых метрик - это рейтинг. Однако он не принимает во внимание популярность фильма. Таким образом, фильм с рейтингом 9 от  10 пользователей будет считаться «лучше», чем фильм с рейтингом 8,9 от 10 000 пользователей. Поэтому стоит использовать взвешенный рейтинг - учитывающий средний рейтинг и количество голосов.

<img src="weighted.png">

* v - количество голосов за фильм

* m - минимальное количество голосов, необходимое для внесения в таблицу

* R - средний рейтинг фильма

* C - средний голос по всем фильмам

m - гиперпараметр. Выберем для него 90 перцентиль - то есть для попадания в таблицу, у фильма должно быть больше голосов, чем у 90 процентов других фильмов.

In [2]:
C = metadata['vote_average'].mean()
print(C)

5.618207215133889


In [3]:
m = metadata['vote_count'].quantile(0.90)
print(m)

160.0


In [4]:
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

(4555, 24)

In [5]:
metadata.columns


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [6]:
from src.features.build_features import calc_weighted_rating


In [7]:
q_movies['score'] = q_movies.apply(lambda x: calc_weighted_rating(x, m, C), axis=1)

In [8]:
q_movies = q_movies.sort_values('score', ascending=False)

In [9]:
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


Ну вот, мы построили простую рейтинговую таблицу на основании взвешенного рейтинга фильмов. Смахивает на кинопоиск, правда?

# Content-based рекомендации

Идея content-based подхода заключается в том, чтобы по истории действий пользователя создать для него вектор его предпочтений в пространстве предметов и рекомендовать товары, близкие к этому вектору.

Попробуем рекомендовать фильмы, похожие на конкретный фильм. Для этого вычислим попарные косинусные расстояния для всех фильмов на основе их сюжетных описаний и порекомендуем фильмы на основе этой метрики.

<img src="content.png">

In [10]:
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

Невозможно вычислить сходство между любыми двумя обзорами в их необработанном виде, поэтому вычислим векторы слов. Как следует из названия, векторы слов - это векторизованное представление слов в документе. Векторы несут семантическое значение. Например, мужчина и король будут иметь векторные представления близко друг к другу, в то время как мужчина и женщина будут иметь представления далеко друг от друга.

Мы вычислим Term Frequency-Inverse Document Frequency (TF-IDF) для каждого документа. Это даст нам матрицу, в которой каждый столбец представляет слово в обзорном словаре (все слова, которые встречаются хотя бы в одном документе), а каждый столбец представляет фильм.

По сути, оценка TF-IDF - это частота встречаемости слова в документе, взвешенная с уменьшением числа документов, в которых оно встречается. Это сделано для уменьшения важности слов, которые часто встречаются в обзорах сюжетов, и, следовательно, их значимости при вычислении окончательной оценки сходства.

TfIdfVectorizer уже реализован в библиотеке scikit. Таким образом, нам нужно проделать следующие шаги:

* импортировать Tfidf из scikit-learn
* убрать стоп-слова, так как они не несут полезной информации
* Замените пропущенные значения пустой строкой
* построить матрицу TF-IDF


In [11]:
metadata['overview'] = metadata['overview'].fillna('')

from src.features.build_features import tfidf_transform
tfidf_matrix = tfidf_transform(metadata)

tfidf_matrix.shape

(45466, 40000)

С помощью этой матрицы мы можем вычислить оценку сходства между двумя фильмами. Мы будем использовать cosine similarity.

<img src="cos.png">

Поскольку мы использовали TF-IDF, вычисление скалярного произведения между каждым вектором напрямую даст вам оценку косинусного сходства. Следовательно, можно использовать linear_kernel() sklearn вместо cosine_similarities(), поскольку он быстрее. Мы получим матрицу размера 45466x45466, где каждый фильм будет вектором-столбцом 1x45466.

In [13]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [14]:
cosine_sim.shape

(45466, 45466)

In [15]:
cosine_sim[1]

array([0.01504121, 1.0000001 , 0.04681953, ..., 0.        , 0.02230244,
       0.00982309], dtype=float32)

Нужно определить функцию, которая принимает на вход название фильма и выводит список из 10 наиболее похожих фильмов. Нужен механизм для определения индекса фильма в metadata по его названию.

In [16]:
import pandas as pd
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

In [19]:
indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

Функция должна выполнять следующие действия:

* Получить индекс фильма по его названию.

* Получить список оценок косинусного сходства для этого конкретного фильма со всеми фильмами. Преобразуйте его в список кортежей, где первый элемент - это его позиция, а второй - оценка сходства.

* Отсортируйте вышеупомянутый список кортежей на основе оценок сходства (второй элемент).

* Получите 10 лучших элементов этого списка. Игнорируйте первый элемент, так как он относится к себе (фильм, наиболее похожий на себя - это он сам).

* Верните заголовки, соответствующие индексам верхних элементов.

In [20]:
from src.models.simple_model import get_recommendations
get_recommendations('The Dark Knight Rises', metadata, indices, cosine_sim)

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
585                                                 Batman
15511                           Batman: Under the Red Hood
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
19792              Batman: The Dark Knight Returns, Part 1
18035                                     Batman: Year One
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object

# Коллаборативная фильтрация

<img src="collaborative.png">

Помимо метаданных фильмов, у нас есть еще один ценный источник информации: данные о рейтингах пользователей. Наша система рекомендаций может порекомендовать фильм, похожий на «Начало (2010)», на основе оценок пользователей. Другими словами, какие еще фильмы получили аналогичные оценки других пользователей? Это был бы пример коллаборативной фильтрации item-item. Таким примером является рекомендация по типу «Пользователям, которым понравился этот элемент, понравились и другие». Мы будем исследовать набор данных ratings.csv, и сформируем векторы оценок пользователей.

В файле links.csv лежит маппинг разных id фильмов. В частности в ratings.csv (который мы будем использовать) используется movieId. В уже знакомом нам metadata используется tmdbId. Чтобы использовать обе таблички, добавим в metadata колонку с movieId - которая по сути является маппингом на колонку metadata['id'] с помощью links.

In [22]:
from src.data.dataset import get_links

links = get_links()

links = links.dropna()

links = links.drop_duplicates(subset=["tmdbId"],keep='first')

id_to_movieId = dict(links[['tmdbId','movieId']].values)

metadata['id'] = pd.to_numeric(metadata['id'], errors='coerce')

metadata = metadata.dropna(subset=['id'])

metadata['movieId'] = metadata['id'].map(id_to_movieId).astype(int)

C:\Users\lucky_man\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [23]:
#Создадим словарь отображения названия фильма в его movieId
title_to_id = dict(zip(metadata.title.tolist(), metadata.movieId.tolist()))

В вашем распоряжении есть два файла - ratings.csv и ratings_small.csv. Второй файлик содержит намного меньше информации, но для быстрой работы давайте использовать его (хотя качество, конечно же, получится хуже). При желании вы можете обучить модель на полном файле рейтингов (ratings.csv).

In [24]:
from src.data.dataset import get_ratings

ratings = get_ratings()

In [25]:
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [26]:
correct_users = ratings.userId.unique()[ratings.groupby('userId')['rating'].count() >= 50]
ratings_f = ratings[ratings.userId.isin(correct_users)]  #Отфильтруйте таблицу - оставьте только те userId, которые проставили не менее 50 оценок

ratings_pivot = pd.pivot_table(ratings_f, index=['movieId'], columns=['userId'], values='rating') #Создайте pivot таблицу из отфильтрованной ratings_f. Индексы - фильмы, колонки - пользователи, значения - рейтинг

In [27]:
ratings_pivot.head()

userId,2,3,4,5,7,8,12,13,15,17,...,655,656,658,659,660,662,664,665,667,671
movieId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,2.0,NaN,...,NaN,NaN,NaN,NaN,2.5,NaN,3.5,NaN,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,4.0,NaN,NaN,NaN,NaN,5.0,NaN,3.0,NaN,NaN
3,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN


##### Matrix Factorization.

Что ж, теперь в нашем распоряжении есть матрица оценок пользователь-айтем. Из математики мы знаем, что любую матрицу можно разложить на произведение трех матриц - например алгоритмом SVD. Но матрицы оценок очень разрежены, 99 % — обычное дело. А SVD не знает, что такое пропуски. Заполнять их средним значением не очень хочется. И в целом, нас не очень интересует матрица сингулярных значений — мы просто хотим получить скрытое представление пользователей и предметов, которое при перемножении будет приближать истинный рейтинг. Можно сразу раскладывать на две матрицы.

<img src="svd.png">

Что же делать с пропусками? Забить на них. Оказалось, что можно успешно обучать приближать рейтинги по метрике RMSE с помощью SGD или ALS, вообще игнорируя пропуски. Первый такой алгоритм — Funk SVD, который придумали в 2006 году в ходе решения соревнования от Netflix.

Но например в задаче рекомендации товаров, мы имеем уже не матрицу оценок, а матрицу некоторых событий. Она будет состоять в основном из нулей и единичек, иногда каких то чисел побольше. Таким образом, у  нас будут присутствовать только положительные примеры. У нас нет примеров товаров, которые человек никогда не купит - получается мы не можем понять, человек не видел товар или он ему не нравится. Таким образом фидбэк от пользователя может быть двух типов:

* Explicit feedback - есть положительные и отрицательные примеры.
* Implicit feedback - есть только положительные.

Так вот, забить на пропуски получается только в случае задачи explicit feedback. В случае implicit можно заполнить пропущенные значения например нулем, и настроить веса в оптимизируемом функционале - низкие для нулей, и повыше для ненулевых ячеек.

Для того чтобы лучше разобраться в математике, можно прочитать например вот [этот пост](https://habr.com/ru/company/yandex/blog/241455/).

<img src="netflix.png">

Алгоритм SVD реализован в библиотеке [Surprise](https://github.com/NicolasHug/Surprise). Далее мы обучим модель и используем ее для прогнозирования рейтингов фильмов, которые данный пользователь, например с 𝑖𝑑 = 2, еще не получил оценку.

In [28]:
#!pip install scikit-surprise

In [36]:
%%time
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_f[['userId','movieId','rating']], reader)

trainset, testset = train_test_split(data, test_size=.25, random_state=42)

from src.models.svd import train_svd
algorithm = train_svd(trainset)

from src.evaluate.evaluate import evaluate_svd
print(evaluate_svd(algorithm, testset))

RMSE: 0.9001
0.9001454029024527
Wall time: 7.56 s


Напишем нашу функцию, которая принимает на вход id пользователя, а на выходе предлагает ему топ 10 фильмов, которые он еще не видел.

In [37]:
from src.models.svd import recommend_svd
user_id = 2
predicted_ratings = recommend_svd(algorithm, user_id, ratings_f, title_to_id)

In [38]:
predicted_ratings

,ratings
movies,
The Godfather,4.570011
On the Waterfront,4.512116
Chinatown,4.498240
Modern Times,4.477477
The Godfather: Part II,4.472773
Fargo,4.463801
Cinema Paradiso,4.423796
It Happened One Night,4.420678
The African Queen,4.396103
